# SI 608 

### Build graph

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from fuzzywuzzy import process

In [2]:
df = pd.read_csv("dblp_graph_conference_v12_new_author.csv")

In [3]:

# exploded_author_ids = df['author_id'].str.split(';').explode()


# unique_author_ids = exploded_author_ids.unique()

# print("Unique Author IDs:", len(unique_author_ids))


In [4]:
# from itertools import combinations
# unique_pairs = set()

# for _, group in df.groupby('id'):
#     all_authors = set(';'.join(group['author_id']).split(';'))
    
#     if len(all_authors) < 2:
#         continue

#     unique_pairs.update(combinations(sorted(all_authors), 2))
# print("Number of unique collaboration pairs:", len(unique_pairs))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097006 entries, 0 to 1097005
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   index            1097006 non-null  int64  
 1   author_id        1097006 non-null  int64  
 2   author_name      1097006 non-null  object 
 3   author_org       1008180 non-null  object 
 4   id               1097006 non-null  int64  
 5   title            1097006 non-null  object 
 6   year             1097006 non-null  int64  
 7   n_citation       1097006 non-null  int64  
 8   doc_type         1097006 non-null  object 
 9   reference_count  1097006 non-null  float64
 10  references       991470 non-null   object 
 11  venue_id         1097006 non-null  int64  
 12  venue_name       1097006 non-null  object 
 13  venue_type       1097006 non-null  object 
 14  doi              939216 non-null   object 
 15  keyword          1097006 non-null  object 
 16  volume           1

In [6]:
df.isnull().sum()

index                    0
author_id                0
author_name              0
author_org           88826
id                       0
title                    0
year                     0
n_citation               0
doc_type                 0
reference_count          0
references          105536
venue_id                 0
venue_name               0
venue_type               0
doi                 157790
keyword                  0
volume              947805
issue              1037011
publisher                0
dtype: int64

In [7]:
df["author_org"] = df["author_org"].fillna("unknown")


# df["reference_count"] = df["reference_count"].fillna(0)

# df["keyword"] = df["keyword"].fillna("unknown")

# df["publisher"] = df["publisher"].fillna("unknown")


Missing doi's do not affect the data and are not filled

Volume and issue are also not filled since there are too many vacancies

#### Title df to Author df

In [8]:
journal_cols = df.columns.to_list()[:3] + df.columns.to_list()[6:]
print(journal_cols)

['index', 'author_id', 'author_name', 'year', 'n_citation', 'doc_type', 'reference_count', 'references', 'venue_id', 'venue_name', 'venue_type', 'doi', 'keyword', 'volume', 'issue', 'publisher']


In [9]:
# rows = []


# for i in range(len(df)):
#     author_names = df["author_name"][i].split(";")
#     author_ids = df["author_id"][i].split(";")
#     author_orgs = str(df["author_org"][i]).split(";")
#     journal_values = {col: df[col][i] for col in journal_cols}

#     for j, author_id in enumerate(author_ids):
#         row = {
#             "index": i,
#             "author_id": author_id,
#             "author_name": author_names[j] if j < len(author_names) else np.nan,
#             "author_org": author_orgs[j] if j < len(author_orgs) else np.nan,
#         }
#         row.update(journal_values)
#         rows.append(row)


# author_df = pd.DataFrame(rows)


In [10]:
author_df=df

In [11]:
# author_df["index"] = author_df["index"] + 1
# author_df["author_id"] = author_df["author_id"].astype(int)
# author_df['reference_count'] = author_df['reference_count'].astype(int)

#### Clean org

In [12]:
author_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097006 entries, 0 to 1097005
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   index            1097006 non-null  int64  
 1   author_id        1097006 non-null  int64  
 2   author_name      1097006 non-null  object 
 3   author_org       1097006 non-null  object 
 4   id               1097006 non-null  int64  
 5   title            1097006 non-null  object 
 6   year             1097006 non-null  int64  
 7   n_citation       1097006 non-null  int64  
 8   doc_type         1097006 non-null  object 
 9   reference_count  1097006 non-null  float64
 10  references       991470 non-null   object 
 11  venue_id         1097006 non-null  int64  
 12  venue_name       1097006 non-null  object 
 13  venue_type       1097006 non-null  object 
 14  doi              939216 non-null   object 
 15  keyword          1097006 non-null  object 
 16  volume           1

In [13]:
author_df.isnull().sum()

index                    0
author_id                0
author_name              0
author_org               0
id                       0
title                    0
year                     0
n_citation               0
doc_type                 0
reference_count          0
references          105536
venue_id                 0
venue_name               0
venue_type               0
doi                 157790
keyword                  0
volume              947805
issue              1037011
publisher                0
dtype: int64

In [14]:
def clean_org(org):
    if "email" in org:
        org = org.rsplit(",", 1)[0]
    return org


author_df.loc[author_df["author_org"] == "nan", "author_org"] = "unknown"
author_df["author_org"] = author_df["author_org"].apply(
    lambda x: x.strip('"').replace("#TAB#", "").strip()
)

author_df["author_org"] = author_df["author_org"].apply(clean_org)

In [15]:
print(author_df["author_org"].value_counts())

author_org
unknown                                                                   153105
Carnegie - Mellon University                                                3996
Stanford, University                                                        2273
Carnegie-Mellon Univ., Pittsburgh, PA, USA                                  1954
Carnegie Mellon University, Pittsburgh Pa                                   1756
                                                                           ...  
IRIT-UPS, Toulouse Cedex, France                                               1
THALES Communications France, Colombes, France                                 1
LIRIS, Lyon 1 Univ., Lyon, France                                              1
INRP, ENS Cachan, Cachan, France                                               1
Archaeological Computing Research Group, University of Southampton, UK         1
Name: count, Length: 243247, dtype: int64


In [16]:
author_df["author_org"].isnull().sum()

0

In [17]:
# organization_names = author_df["author_org"].unique()

# standardized_dict = {}
# threshold = 85

# for org_name in organization_names:
#     if org_name in standardized_dict:
#         continue
#     if org_name == "unknown":
#         standardized_dict[org_name] = org_name
#         continue
#     if not org_name.strip():
#         continue

#     matched_name = None
#     for std_name in standardized_dict.keys():
#         if process.extractOne(org_name, [std_name])[1] >= threshold:
#             matched_name = std_name
#             break

#     if matched_name:
#         standardized_dict[org_name] = matched_name
#     else:
#         standardized_dict[org_name] = org_name

# author_df["standard_org"] = author_df["author_org"].map(standardized_dict)

In [18]:
# author_df["standard_org"].isnull().sum()

In [19]:
# print(author_df.loc[author_df["standard_org"].isna(), "author_org"])

In [20]:
# author_df["standard_org"].value_counts()

In [21]:
# author=author_df.drop(columns=["author_org"])
# author["standard_org"].fillna("unknown", inplace=True)

#### Other paper related attributes

In [22]:
author=author_df
author.rename(columns={"author_org":"standard_org"}, inplace=True)

In [23]:
author["publisher"].value_counts()

publisher
IEEE                                                                                               376014
ACM                                                                                                164336
Springer, Berlin, Heidelberg                                                                       157110
unknown                                                                                            108753
Springer, Cham                                                                                      63753
                                                                                                    ...  
IEEE EMBS Int Conf Biomed Health Inform                                                                 1
The 26th International Conference on Software Engineering and Knowledge Engineering (SEKE 2014)         1
The 29th Pacific Asia Conference on Language, Information and Computation                               1
ACADEMIC PRESS LTD- ELSEVIER SCIENCE

In [24]:
author.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097006 entries, 0 to 1097005
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   index            1097006 non-null  int64  
 1   author_id        1097006 non-null  int64  
 2   author_name      1097006 non-null  object 
 3   standard_org     1097006 non-null  object 
 4   id               1097006 non-null  int64  
 5   title            1097006 non-null  object 
 6   year             1097006 non-null  int64  
 7   n_citation       1097006 non-null  int64  
 8   doc_type         1097006 non-null  object 
 9   reference_count  1097006 non-null  float64
 10  references       991470 non-null   object 
 11  venue_id         1097006 non-null  int64  
 12  venue_name       1097006 non-null  object 
 13  venue_type       1097006 non-null  object 
 14  doi              939216 non-null   object 
 15  keyword          1097006 non-null  object 
 16  volume           1

In [25]:
author['venue_name'].value_counts()

venue_name
National Conference on Artificial Intelligence                                     15827
Computer Vision and Pattern Recognition                                            15516
International Conference on Acoustics, Speech, and Signal Processing               13433
International Conference on Robotics and Automation                                12741
International Conference on Image Processing                                       11525
                                                                                   ...  
International Symposium on Power Electronics for Distributed Generation Systems        1
IEEE International Conference on Circuits and Systems for Communications               1
European Conference on Radiation and Its Effects on Components and Systems             1
Self-Sustaining Systems                                                                1
International Conference on Software Technology and Engineering                        1
Name: coun

### Split train and validation

In [26]:
most_recent_indices = author.groupby("author_id")["year"].idxmax()

valid = author.loc[most_recent_indices]

train = author.drop(most_recent_indices)

In [27]:
valid.shape

(29694, 19)

In [28]:
train.shape

(1067312, 19)

#### Build the graph

In [29]:
train.columns

Index(['index', 'author_id', 'author_name', 'standard_org', 'id', 'title',
       'year', 'n_citation', 'doc_type', 'reference_count', 'references',
       'venue_id', 'venue_name', 'venue_type', 'doi', 'keyword', 'volume',
       'issue', 'publisher'],
      dtype='object')

In [30]:
train['author_id'].value_counts()   

author_id
2125104194    796
2121069363    701
296576950     694
2168417342    661
2121939561    630
             ... 
2689927708      4
2665205769      4
2294120791      4
2590407387      4
2937986853      4
Name: count, Length: 29694, dtype: int64

In [31]:
citation_grouped= train.groupby("author_id")['n_citation'].sum().reset_index(name="total_citation")
train=train.merge(citation_grouped, on="author_id", how="left")

paper_grouped= train.groupby("author_id")['index'].count().reset_index(name="total_paper")
train=train.merge(paper_grouped, on="author_id", how="left")

reference_grouped= train.groupby("author_id")['reference_count'].mean().reset_index(name="avg_reference")
train=train.merge(reference_grouped, on="author_id", how="left")



In [32]:
def merge_keywords(keywords):
    all_keywords = set()
    for kw in keywords:
        all_keywords.update(kw.split(';'))
    return all_keywords


author_keywords = train.groupby("author_id")["keyword"].apply(merge_keywords).reset_index(name='total_keyword')
train=train.merge(author_keywords, on="author_id", how="left")

In [33]:
train.head()

,index,author_id,author_name,standard_org,id,title,year,n_citation,doc_type,reference_count,...,venue_type,doi,keyword,volume,issue,publisher,total_citation,total_paper,avg_reference,total_keyword
0,0,2117665592,G. Earl,"Archaeological Computing Research Group, Unive...",1674,A methodology for the physically accurate visu...,2011,1,Conference,15.0,...,C,https://doi.org/10.2312/VAST/VAST11/137-144,Statue;Engineering drawing;Virtual reconstruct...,NaN,NaN,Eurographics Association,59,7,10.285714,"{Artificial intelligence, Visualization, Art, ..."
1,1,2101415595,Dipankar Das,Department of Computer Science and Engineering...,9415,Identifying Psychological Theme Words from Emo...,2013,0,Conference,4.0,...,C,https://doi.org/10.1007/978-3-319-03844-5_71,Sadness;Rule-based system;Ranking;Disgust;Sham...,NaN,NaN,"Springer, Cham",480,82,9.463415,"{Visualization, Real-time computing, Online di..."
2,2,123621675,Serge Sharoff,unknown,15548,Cleaneval: a Competition for Cleaning Web Pages,2008,66,Conference,3.0,...,C,NaN,Web design;Web development;World Wide Web;Web ...,NaN,NaN,unknown,451,39,7.897436,"{Web design, CAD, Semantic Web Stack, Knowledg..."
3,3,2137964017,Lidija Comic,unknown,16022,Algorithms for the Construction of Digital Con...,2007,0,Conference,4.0,...,C,NaN,Convexity;Computer science;Fuzzy logic;Algorit...,NaN,NaN,unknown,53,14,14.142857,"{Duality (mathematics), Knowledge representati..."
4,4,2148789325,Prudence W. H. Wong,#N# University of Liverpool#N#,20175,An 8/3 Lower Bound for Online Dynamic Bin Packing,2012,6,Conference,15.0,...,C,https://doi.org/10.1007/978-3-642-35261-4_8,Online algorithm;Discrete mathematics;Combinat...,NaN,NaN,"Springer, Berlin, Heidelberg",574,55,14.563636,"{Linear function, Real-time computing, Interva..."


In [34]:
G_train = nx.Graph()

In [35]:
for index, row in train.iterrows():
    author_id = row['author_id']
    if not G_train.has_node(author_id):
        G_train.add_node(
            author_id,
            name=row['author_name'],
            org=row['standard_org'],
            total_citation=row["total_citation"], 
            total_paper=row["total_paper"],
            avg_reference=row["avg_reference"],     
            keywords=row["total_keyword"],   
        )

In [36]:

for paper_id, group in train.groupby('id'):

    authors = group['author_id'].tolist()
    id = group['id'].iloc[0]
    title = group['title'].iloc[0]
    year = group['year'].iloc[0]
    venue_id = group['venue_id'].iloc[0]
    venue_name = group['venue_name'].iloc[0]
    publisher = group['publisher'].iloc[0]
    

    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            author1 = authors[i]
            author2 = authors[j]
            
            if G_train.has_edge(author1, author2):
                G_train[author1][author2]['weight'] += 1
                G_train[author1][author2]['papers'].append({'id': id, 'title': title, 'year': year,'venue_id':venue_id,'venue_name':venue_name,'publisher':publisher})
            else:
                G_train.add_edge(
                    author1,
                    author2,
                    weight=1,
                    papers=[{'id': id, 'title': title, 'year': year,'venue_id':venue_id,'venue_name':venue_name,'publisher':publisher}],
                )


### Network Analysis

In [38]:
print("# components: ",nx.number_connected_components(G_train))

nodes_num=len(G_train.nodes())
print('# nodes: ',nodes_num)

edges_num=len(G_train.edges())
print('# edge: ',edges_num)

largest_component = max(nx.connected_components(G_train), key=len)
percent_lc=round(len(largest_component)/nodes_num*100,2)
print("%_largest component: ",percent_lc)

# average degree
degree_dict=dict(nx.degree(G_train))
avg_d=sum(degree_dict.values())/len(G_train.nodes())
print("average degree: ",avg_d)

# average clustering coefficient
avg_cc=nx.average_clustering(G_train)
print("average clustering coefficient: ",avg_cc)

# components:  1095
# nodes:  29694
# edge:  153857
%_largest component:  94.4
average degree:  10.362834242607935
average clustering coefficient:  0.37782319359035915
